# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,nova sintra,14.8667,-24.7167,23.51,75,0,6.19,CV,1712607310
1,1,munirabad,15.3333,76.3333,29.75,40,88,3.48,IN,1712607035
2,2,cabo san lucas,22.8909,-109.9124,24.61,68,75,1.79,MX,1712607311
3,3,college,64.8569,-147.8028,0.97,71,40,2.06,US,1712607065
4,4,lerwick,60.1545,-1.1494,8.00,79,100,7.26,GB,1712607144


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
#%%capture --no-display

humidity = city_data_df["Humidity"].astype(int)

# Configure the map
map_plot_humid = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriImagery",
    frame_width = 800,
    frame_height = 600,
    size = "Distance",
    scale = 0.5,
    color = "City"
)

# Display the map plot
map_plot_humid

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [6]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df

# Apply multiple filtering conditions without reassigning the DataFrame
ideal_weather_df = city_data_df.loc[
    (ideal_weather_df["Max Temp"] < 27) & 
    (ideal_weather_df["Max Temp"] > 21) &
    (ideal_weather_df["Wind Speed"] < 4.5) &
    (ideal_weather_df["Cloudiness"] == 0)
]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
7,7,lashio,22.9333,97.7500,21.13,46,0,1.10,MM,1712607167
192,192,old town,50.0870,14.4202,21.04,62,0,3.09,CZ,1712607342
214,214,claremore,36.3126,-95.6161,26.45,46,0,3.09,US,1712607119
303,303,al jawf,29.5000,38.7500,21.78,20,0,2.82,SA,1712607357
307,307,stephenville,32.2207,-98.2023,26.54,60,0,3.60,US,1712607316
311,311,hamilton,39.1834,-84.5333,24.99,50,0,4.12,US,1712607075
374,374,independence,39.0911,-94.4155,21.69,36,0,4.12,US,1712607333
432,432,fredericksburg,38.3032,-77.4605,22.95,35,0,4.12,US,1712607261
451,451,rampur,28.8167,79.0333,23.39,11,0,0.89,IN,1712607386


### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[["City", "Country", "Lat", "Lng"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Hotel Name
7,lashio,MM,22.9333,97.7500,
192,old town,CZ,50.0870,14.4202,
214,claremore,US,36.3126,-95.6161,
303,al jawf,SA,29.5000,38.7500,
307,stephenville,US,32.2207,-98.2023,
311,hamilton,US,39.1834,-84.5333,
374,independence,US,39.0911,-94.4155,
432,fredericksburg,US,38.3032,-77.4605,
451,rampur,IN,28.8167,79.0333,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [8]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()

Starting hotel search
lashio - nearest hotel: Golden Kaniri Hotel
old town - nearest hotel: Terasa U Prince
claremore - nearest hotel: La Quinta
al jawf - nearest hotel: No hotel found
stephenville - nearest hotel: No hotel found
hamilton - nearest hotel: North Vista Manor
independence - nearest hotel: The Inn at Ophelia’s
fredericksburg - nearest hotel: Kenmore Inn
rampur - nearest hotel: تمنا ہوٹل اور ریستوران


,City,Country,Lat,Lng,Hotel Name
7,lashio,MM,22.9333,97.7500,Golden Kaniri Hotel
192,old town,CZ,50.0870,14.4202,Terasa U Prince
214,claremore,US,36.3126,-95.6161,La Quinta
303,al jawf,SA,29.5000,38.7500,No hotel found
307,stephenville,US,32.2207,-98.2023,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [9]:
#%%capture --no-display

# Configure the map plot
map_plot_hotel = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color="City",
    alpha=.75,
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
map_plot_hotel

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)